# RNA-Seq analysis of T24, T1, T0 samples 


In [2]:
suppressMessages(library(DESeq2))
suppressMessages(library(ggplot2))
suppressMessages(library(cowplot))

In [2]:
cbbPalette <- c("#999999", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7")


In [3]:
annotations <- read.table(file.path('..', 'data', 'annotations', 'hg38_gene_names_stripped.tsv'), 
                          header=F, 
                          col.names = c('gene_id', 'gene_name', 'gene_type'))
rownames(annotations) <- annotations$gene_id

readcounts.dir <- file.path('..', 'data', 'read_counts', 'byExon', 'RNA_seq')
results.dir <- file.path('..', 'results', 'RNA_seq')
design.file <-  file.path('..', 'data', 'design_files', 'rna_seq_design.tsv')

## Suffix of htseq-count output
counts.suffix <- '.exon.counts.tsv'

write_results <- function(df, prefix){
  df<- as.data.frame(df)
  df <- df[order(df$padj),]
  df$gene_name <- annotations[rownames(df),]$gene_name
  df.sig <- subset(df, padj<0.05)
  df.sig.up <- subset(df.sig, log2FoldChange>0)
  df.sig.down <- subset(df.sig, log2FoldChange<0)
  write.table(df, file = file.path(results.dir, 
                                   paste(prefix, 'tsv', sep='.')), sep = '\t')

  write.table(df.sig, file = file.path(results.dir, 
                                   paste(prefix, 'sig', 'tsv', sep='.')), sep = '\t')
  write.table(df.sig.up,  file = file.path(results.dir,
                                       paste(prefix, 'sig', 'up', 'tsv', sep='.')), sep = '\t')
  write.table(df.sig.down,  file = file.path(results.dir,
                                         paste(prefix, 'sig', 'down', 'tsv', sep='.')), sep = '\t')
  return (df.sig)
}

PCAplot <- function(rld){
  data <- plotPCA(rld, intgroup=c('Cell.line', 'Time'), returnData=TRUE, ntop=1210000)
  percentVar <- round(100 * attr(data, 'percentVar'))
  ggplot(data, aes(PC1, PC2, color=Time, shape=CellLine)) +
    geom_point(size=3) +
    theme_bw(base_size = 16) + 
    xlab(paste0('PC1: ',percentVar[1],'% variance')) +
    ylab(paste0('PC2: ',percentVar[2],'% variance')) +
    coord_fixed() +
    scale_color_manual("Time", values = c('T0' = cbbPalette[6], 
                                          'T1' = cbbPalette[7],                                         
                                          'T24' = cbbPalette[4])) +
    theme(aspect.ratio=1, plot.title = element_text(hjust = 0.5))
}

PCAplot.cellLine <- function(rld){
  data <- plotPCA(rld, intgroup=c('Time'), returnData=TRUE)
  percentVar <- round(100 * attr(data, 'percentVar'))
  ggplot(data, aes(PC1, PC2, color=Time)) +
    geom_point(size=3) +
    theme_bw(base_size = 16) + 
    xlab(paste0('PC1: ',percentVar[1],'% variance')) +
    ylab(paste0('PC2: ',percentVar[2],'% variance')) +
    coord_fixed() +
    scale_color_manual("Time", values = c('T0' = cbbPalette[6], 
                                          'T1' = cbbPalette[7], 
                                          'T24' = cbbPalette[4])) + 
    theme(aspect.ratio=1, plot.title = element_text(hjust = 0.5))
}


design.info <- read.table(design.file, header=T, stringsAsFactors=FALSE)
design.info$Time <- as.factor(design.info$Time)
design.info$Cell.line <- as.factor(design.info$Cell_line)
files <- paste(design.info$SampleFile, counts.suffix, sep='')
sampleName <- design.info$SampleName
Time <- design.info$Time
Cell.line <- design.info$Cell.line
group <- factor(paste0(Cell.line, '_', Time))
sampleTable <- data.frame(sampleName = sampleName, 
                          fileName = files, 
                          Cell.line = Cell.line,  
                          Time = Time, 
                          group = group)
ddsHTSeq <- DESeqDataSetFromHTSeqCount(sampleTable = sampleTable, 
                                       directory = readcounts.dir,
                                       design = ~ group)
rownames(ddsHTSeq) <- gsub('\\.[0-9]+', '', rownames(ddsHTSeq))
ddsHTSeq <- ddsHTSeq[ rowSums(counts(ddsHTSeq)) > 1, ]
dds <- DESeq(ddsHTSeq)

resultsNames(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1] "Intercept"                 "group_U251_T1_vs_U251_T0" 
[3] "group_U251_T24_vs_U251_T0" "group_U343_T0_vs_U251_T0" 
[5] "group_U343_T1_vs_U251_T0"  "group_U343_T24_vs_U251_T0"

In [1]:
design.file

ERROR: Error in eval(expr, envir, enclos): object 'design.file' not found


# All T1 vs T0

In [4]:
All.res.T1vsT0 <- results(dds, 
                          contrast = list(c('group_U251_T1_vs_U251_T0', 
                                            'group_U343_T1_vs_U251_T0'), 
                                          'group_U343_T0_vs_U251_T0'), 
                          listValues = c(1, -1))
write_results(All.res.T1vsT0, 'All.RNASeq.res.T1vsT0')

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
ENSG00000162783,3859.0501,1.1523116,0.14304024,8.055857,7.892411e-16,1.185282e-11,IER5
ENSG00000196670,879.1671,-1.1075250,0.16077484,-6.888671,5.631612e-12,4.228778e-08,ZFP62
ENSG00000125740,127.1540,2.1548858,0.32128571,6.707070,1.985710e-11,9.564706e-08,FOSB
ENSG00000181026,6536.5288,0.9609583,0.14405866,6.670604,2.547531e-11,9.564706e-08,AEN
ENSG00000081019,1250.8434,-1.0362656,0.16010273,-6.472504,9.639176e-11,2.301739e-07,RSBN1
ENSG00000101665,299.3777,1.7419188,0.26984371,6.455288,1.080134e-10,2.301739e-07,SMAD7
ENSG00000155090,2370.5774,1.0224928,0.15819502,6.463495,1.023116e-10,2.301739e-07,KLF10
ENSG00000196263,349.2331,-1.4600091,0.22684817,-6.436063,1.226123e-10,2.301739e-07,ZNF471
ENSG00000164011,428.0047,-0.9660298,0.15594801,-6.194563,5.844698e-10,9.752853e-07,ZNF691


# All T24 vs T0

In [5]:
All.res.T24vsT0 <- results(dds, 
                           contrast = list(c('group_U251_T24_vs_U251_T0', 
                                             'group_U343_T24_vs_U251_T0'), 
                                           'group_U343_T0_vs_U251_T0'), 
                           listValues = c(1, -1))
write_results(All.res.T24vsT0, 'All.RNASeq.res.T24vsT0')

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
ENSG00000116237,12958.2615,-1.3188815,0.08353894,-15.78763,3.785665e-56,7.396433e-52,ICMT
ENSG00000029153,4111.5523,-1.5607440,0.11988306,-13.01889,9.555386e-39,9.334656e-35,ARNTL2
ENSG00000077684,3670.5191,-1.6146395,0.13386599,-12.06161,1.684543e-33,1.097087e-29,JADE1
ENSG00000105559,757.8621,2.7706814,0.23413910,11.83348,2.620329e-32,1.279900e-28,PLEKHA4
ENSG00000108840,1322.5406,2.2836299,0.19347859,11.80301,3.765908e-32,1.471566e-28,HDAC5
ENSG00000146411,5929.1014,2.2617788,0.19261515,11.74248,7.719288e-32,2.325789e-28,SLC2A12
ENSG00000167081,1817.6761,-1.7059581,0.14536102,-11.73601,8.332747e-32,2.325789e-28,PBX3
ENSG00000143816,727.2148,2.9641297,0.25762071,11.50579,1.233551e-30,3.012641e-27,WNT9A
ENSG00000128973,4417.0014,-1.4951788,0.13022710,-11.48132,1.637637e-30,3.555129e-27,CLN6


# All T24 vs T1

In [6]:
All.res.T24vsT1 <- results(dds, 
                           contrast = list(c('group_U251_T24_vs_U251_T0', 
                                             'group_U343_T24_vs_U251_T0'), 
                                           c('group_U251_T1_vs_U251_T0',
                                             'group_U343_T1_vs_U251_T0')), 
                           listValues = c(1, -1))
write_results(All.res.T24vsT1, 'All.RNASeq.res.T24vsT1')

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
ENSG00000116237,12958.2615,-1.4327097,0.07875399,-18.19222,5.948373e-74,1.195801e-69,ICMT
ENSG00000105559,757.8621,3.1764465,0.21349402,14.87839,4.553503e-50,4.576954e-46,PLEKHA4
ENSG00000146411,5929.1014,2.6090809,0.18081069,14.42990,3.355481e-47,2.248508e-43,SLC2A12
ENSG00000029153,4111.5523,-1.6136551,0.11299199,-14.28115,2.868434e-46,1.441603e-42,ARNTL2
ENSG00000128973,4417.0014,-1.6573802,0.12273521,-13.50371,1.487041e-41,5.978798e-38,CLN6
ENSG00000167081,1817.6761,-1.8235252,0.13724176,-13.28696,2.755642e-40,9.232778e-37,PBX3
ENSG00000171848,14940.3322,-2.9904670,0.22527120,-13.27496,3.234293e-40,9.288428e-37,RRM2
ENSG00000124145,5902.0235,-1.5124376,0.11651308,-12.98084,1.571488e-38,3.948952e-35,SDC4
ENSG00000142856,1670.9098,-1.6443723,0.13307770,-12.35648,4.493963e-35,1.003802e-31,ITGB3BP
